In [4]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

class EncryptionManager:
    def __init__(self):
        self.key = os.urandom(32)
        self.iv = os.urandom(16)
        aesContext = Cipher(algorithms.AES(self.key), modes.CBC(self.iv), backend=default_backend())
        self.encryptor = aesContext.encryptor()
        self.decryptor = aesContext.decryptor()
        self.padder = padding.PKCS7(128).padder()
        self.unpadder = padding.PKCS7(128).unpadder()
        
    def update_encryptor(self, plaintext):
        return self.encryptor.update(self.padder.update(plaintext))
    
    def finalize_encryptor(self):
        return self.encryptor.update(self.padder.finalize()) + self.encryptor.finalize()
    
    def update_decryptor(self, ciphertext):
        return self.unpadder.update(self.decryptor.update(ciphertext))
    
    def finalize_decryptor(self):
        return self.unpadder.update(self.decryptor.finalize()) + self.unpadder.finalize()
    
    def encrypt_message(self, message):
        encryptor = Cipher(algorithms.AES(self.key), modes.CBC(self.iv), backend=default_backend()).encryptor()
        padder = padding.PKCS7(128).padder()
        padded_message = padder.update(message)
        padded_message += padder.finalize()
        ciphertext = encryptor.update(padded_message)
        ciphertext += encryptor.finalize()
        return ciphertext
    
    def decrypt_message(self, message):
        decryptor = Cipher(algorithms.AES(self.key), modes.CBC(self.iv), backend=default_backend()).decryptor()
        unpadder = padding.PKCS7(128).unpadder()
        padded_message = decryptor.update(message)
        padded_message += decryptor.finalize()
        message = unpadder.update(padded_message)
        message += unpadder.finalize()
        return message
    
manager = EncryptionManager()

plaintexts = [
    b"SHORT",
    b"MEDIUM MEDIUM MEDIUM",
    b"LONG LONG LONG LONG LONG LONG"
]

ciphertexts = []

for m in plaintexts:
    ciphertexts.append(manager.update_encryptor(m))
ciphertexts.append(manager.finalize_encryptor())

for c in ciphertexts:
    print("Recovered", manager.update_decryptor(c))
print("Recovered", manager.finalize_decryptor())

plain_messages = [
    b"SHORT",
    b"MEDIUM MEDIUM MEDIUM",
    b"LONG LONG LONG LONG LONG LONG"
]

ms_manager = EncryptionManager()
cipher_messages = []

for m in plain_messages:
    cipher_messages.append(ms_manager.encrypt_message(m))
    
for c in cipher_messages:
    print("Recovered: ", ms_manager.decrypt_message(c))
    

Recovered b''
Recovered b''
Recovered b'SHORTMEDIUM MEDIUM MEDIUMLONG LO'
Recovered b'NG LONG LONG LON'
Recovered b'G LONG'
Recovered:  b'SHORT'
Recovered:  b'MEDIUM MEDIUM MEDIUM'
Recovered:  b'LONG LONG LONG LONG LONG LONG'


In [6]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os

class EncryptionManager:
    def __init__(self):
        self.key = os.urandom(32)
        self.iv = os.urandom(16)
        aesContext = Cipher(algorithms.AES(self.key), modes.CTR(self.iv), backend=default_backend())
        self.encryptor = aesContext.encryptor()
        self.decryptor = aesContext.decryptor()
        
    def updateEncryptor(self, plaintext):
        return self.encryptor.update(plaintext)
    
    def finalizeEncryptor(self):
        return self.encryptor.finalize()
    
    def updateDecryptor(self, ciphertext):
        return self.decryptor.update(ciphertext)
    
    def finalizeDecryptor(self):
        return self.decryptor.finalize()
    
manager = EncryptionManager()

plaintexts = [
    b"SHORT",
    b"MEDIUM MEDIUM MEDIUM",
    b"LONG LONG LONG LONG LONG LONG"
]

ciphertexts = []

for m in plaintexts:
    ciphertexts.append(manager.updateEncryptor(m))
#ciphertexts.append(manager.finalizeEncryptor())

for c in ciphertexts:
    print("Recovered", manager.updateDecryptor(c))
#print("Recovered", manager.finalizeDecryptor())

Recovered b'SHORT'
Recovered b'MEDIUM MEDIUM MEDIUM'
Recovered b'LONG LONG LONG LONG LONG LONG'


In [ ]:
# RSA test
import gmpy2, os, binascii
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

def simple_rsa_encrypt(m, publickey):
    numbers = publickey.public_numbers()
    return gmpy2.powmod(m, numbers.e, numbers.n)

def simple_rsa_decrypt(m, privatekey):
    numbers = privatekey.private_numbers()
    return gmpy2.powmod(m, numbers.d, numbers.public_numbers.n)

def int_to_bytes(i):
    i = int(i)
    return i.to_bytes((i.bit_length()+7)//8, byteorder="big")

def bytes_to_int(b):
    return int.from_bytes(b, byteorder="big")

def main():
    public_key_file = None
    private_key_file = None
    public_key = None
    private_key = None
    while True:
        print("Simple RSA Crypto")
        print("--------------------")
        print("\tprivate key file: {}".format(private_key_file))
        print("\tpublic key file: {}".format(public_key_file))
        print("\t1. Encrypt Message.")
        print("\t2. Decrypt Message.")
        print("\t3. Load public key file.")
        print("\t4. Load private key file.")
        print("\t5. Create and load new public and private key files.")
        print("\t6. Quit.\n")
        choice = input(" >> ")
        if choice == "1":
            if not public_key:
                print("\nNo public key loaded\n")
            else:
                message = input("\nPlaintext: ").encode()
                message_as_int = bytes_to_int(message)
                cipher_as_int = simple_rsa_encrypt(message_as_int, public_key)
                cipher = int_to_bytes(cipher_as_int)
                print("\nCiphertext (hexlified): {}\n".format(binascii.hexlify(cipher)))
        elif choice == "2":
            if not private_key:
                print("\nNo private key loaded\n")
            else:
                cipher_hex = input("\nCiphertext (hexlified): ").encode()
                cipher = binascii.unhexlify(cipher_hex)
                cipher_as_int = bytes_to_int(cipher)
                message_as_int = simple_rsa_decrypt(cipher_as_int, private_key)
                message = int_to_bytes(message_as_int)
                print("\nPlaintext: {}\n".format(message))
        elif choice == "3":
            public_key_file_temp = input("\nEnter public key file: ")
            if not os.path.exists(public_key_file_temp):
                print("File {} does not exist.".format(public_key_file_temp))
            else:
                with open(public_key_file_temp, "rb") as public_key_file_object:
                    public_key = serialization.load_pem_public_key(public_key_file_object.read(), backend=default_backend())
                    public_key_file = public_key_file_temp
                    print("\nPublic key file loaded.\n")
                    
                    private_key_file = None
                    private_key = None
                    
        elif choice == "4":
            private_key_file_temp = input("\nEnter private key file: ")
            if not os.path.exists(private_key_file_temp):
                print("File {} does not exists.".format(private_key_file_temp))
            else:
                with open(private_key_file_temp) as private_key_file_object:
                    private_key = serialization.load_pem_private_key(private_key_file_object.read(), backend=default_backend(), password=None)
                    private_key_file = private_key_file_temp
                    print("\nPrivate key file loaded.\n")
                    
                    public_key = private_key.public_key()
                    public_key_file = None
                    
        elif choice == "5":
            private_key_file_temp = input("\nEnter a file name for new private key: ")
            public_key_file_temp = input("\nEnter a file name for a new public key: ")
            if os.path.exists(private_key_file_temp) or os.path.exists(public_key_file_temp):
                print("File already exists.")
            else:
                with open(private_key_file_temp, "wb+") as private_key_file_obj:
                    with open(public_key_file_temp, "wb+") as public_key_file_obj:
                        private_key = rsa.generate_private_key(
                            public_exponent =65537,
                            key_size =2048,
                            backend = default_backend()
                        )
                        public_key = private_key.public_key()
                        private_key_bytes = private_key.private_bytes(
                            encoding=serialization.Encoding.PEM,
                            format=serialization.PrivateFormat.TraditionalOpenSSL,
                            encryption_algorithm=serialization.NoEncryption()
                        )
                        private_key_file_obj.write(private_key_bytes)
                        public_key_bytes = public_key.public_bytes(
                            encoding=serialization.Encoding.PEM,
                            format=serialization.PublicFormat.SubjectPublicKeyInfo
                        )
                        public_key_file_obj.write(public_key_bytes)

                        public_key_file = None
                        private_key_file = private_key_file_temp
        elif choice == '6':
            print("\n\nTerminating. This program will self destruct in 5 seconds.\n")
            break
        else:
            print("\n\nUnknown option {}.\n".format(choice))

if __name__ == '__main__':
    main()

Simple RSA Crypto
--------------------
	private key file: None
	public key file: None
	1. Encrypt Message.
	2. Decrypt Message.
	3. Load public key file.
	4. Load private key file.
	5. Create and load new public and private key files.
	5. Quit.

 >> 5

Enter a file name for new private key: t_private

Enter a file name for a new public key: t_public
Simple RSA Crypto
--------------------
	private key file: t_private
	public key file: None
	1. Encrypt Message.
	2. Decrypt Message.
	3. Load public key file.
	4. Load private key file.
	5. Create and load new public and private key files.
	5. Quit.

 >> 1

Plaintext: abcde

Ciphertext (hexlified): b'0fc8252ab432472d6ac9686c2c7bffc476ec48e9a53e372c230ece270794fe1da36cfd8d9aa42244fe0dda25e5b2df69ef97cb56256d9ffca15f557929f160adcbcd830ea32ac3b1ce6366901104d419da2b303b384217f153d51a4dbefd67fc744b196ad50b8b663c30ebfefd3095dde183680b3b8d45f70fb6594ccece5deb3c7fc7b992fdf6538db313127bf34391600d5960c02e21cdbf59299c868660abaaf6e103b2cef92169ec8472a7